In [ ]:
import requests
import io
import os.path
import pandas as pd
from PIL import Image
import random
import requests
import os
import numpy as np
import keras
from keras import layers
import matplotlib.pyplot as plt
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
##Functions
#Create Sub-folders for each category
def Create_folder_by_value(value,path):
    folder = path+'/{}/'.format(value)
    Already_created = os.path.isdir(folder)
    if (not Already_created):
        os.mkdir(folder)
    return folder

#Save photos in subfolder
def Save_photos(value,photos_id,path):
    for x in photos_id:
        r = requests.get('https://api.isic-archive.com/api/v2/images/{}/'.format(x))
        url = r.json()['files']['full']['url']
        r2 = requests.get(url)
        imageBytes = r2._content
        image = Image.open(io.BytesIO(imageBytes))
        image.save(path+'/{}/'.format(value)+'{}'.format(x)+'.jpeg')

#Train, Test, Split
def Keras_train_split(image_path,image_size,batch_size):
    train_ds, val_ds = keras.utils.image_dataset_from_directory(
    image_path,
    validation_split=0.2,
    subset="both",
    seed=188,
    image_size=image_size,
    batch_size=batch_size,
)
    return train_ds,val_ds

#Image augmentation layer
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

#Model Structure
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = data_augmentation(inputs)
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


#Score new image
def Dermal_Cancer_Score(model_path:str,new_image_path:str):
    model = keras.models.load_model(model_path)
    img = Image.open(new_image_path)
    img_array = keras.utils.img_to_array(img)
    img_array = keras.ops.expand_dims(img_array,0)
    predictions = model.predict(img_array)
    score = float(keras.ops.sigmoid(predictions[0][0]))
    return score

In [ ]:
#Define Parameters
random.seed(188)
image_path = os.getcwd()+'/images'
image_size = (180, 180)
batch_size = 128
epochs = 25

#Create Folder for categories and download corresponding photos for each category
df= pd.read_csv('challenge-2020-training_metadata_2024-03-08.csv')
for value in df['benign_malignant'].unique():
    Create_folder_by_value(value,image_path)
    photos_id= df[df['benign_malignant']== value]['isic_id']
    Save_photos(value,photos_id,image_path)

#Train, Test Split defining Image size for the model
train_ds, val_ds = Keras_train_split(image_path,image_size,batch_size)

#Run Model
model = make_model(input_shape=image_size + (3,), num_classes=2)
callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),]
model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

#Save Model
model.save('cancer_detect_m1.keras')


#Score new Image
new_image_path = os.getcwd()+'/new_images/ISIC_0514672.jpeg'
Dermal_Cancer_Score('cancer_detect_m1.keras',)
